<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/StockAndNewsAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

!pip install pandas-datareader

In [0]:
import urllib
import json
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as dr
import numpy as np
from datetime import datetime, date

In [29]:
def get_data(symbol, start, end):
  # Use IEX API for stats so we get all info we need  IEX Cloud consol 
  https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=symbol,companyName,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent
  # From here we can get the following information
  """
  symbol	string	
  companyName	string	Company name of the security
  marketcap	number	Market cap of the security calculated as shares outstanding * previous day close.
  week52high	number	
  week52low	number	
  week52change	number	Percentage change
  sharesOutstanding	number	Number of shares outstanding as the difference between issued shares and treasury shares. Investopedia
  avg30Volume	number	Average 30 day volume
  avg10Volume	number	Average 10 day volume
  float	number	Returns the annual shares outstanding minus closely held shares.
  employees	number	
  ttmEPS	number	Trailing twelve month earnings per share. Investopedia
  ttmDividendRate	number	Trailing twelve month dividend rate per share
  dividendYield	number	The ratio of trailing twelve month dividend compared to the previous day close price. The dividend yield is represented as a percentage calculated as (ttmDividendRate) / (previous day close price) Investopedia
  nextDividendDate	string	Expected ex date of the next dividend
  exDividendDate	string	Ex date of the last dividend
  nextEarningsDate	string	Expected next earnings report date
  peRatio	number	Price to earnings ratio calculated as (previous day close price) / (ttmEPS)
  beta	number	Beta is a measure used in fundamental analysis to determine the volatility of an asset or portfolio in relation to the overall market. Levered beta calculated with 1 year historical data and compared to SPY.
  day200MovingAvg	number	
  day50MovingAvg	number	
  maxChangePercent	number	
  year5ChangePercent	number	
  year2ChangePercent	number	
  year1ChangePercent	number	
  ytdChangePercent	number	
  month6ChangePercent	number	
  month3ChangePercent	number	
  month1ChangePercent	number	
  day30ChangePercent	number	
  day5ChangePercent	number	
  
  
  """
  
  
  data =  dr.DataReader(symbol, 'iex', start, end)[['close']]
  return data.rename(columns={'close': symbol})

SyntaxError: ignored

<h2>Authenticate User </h2>

In [0]:
from google.colab import auth
auth.authenticate_user()

<h3> Loading Nasdaq and Nyse shares </h3>

In [31]:
!gsutil cp gs://datascience-bucket-mm/nyse-companylist.csv /tmp/nyse.csv
!gsutil cp gs://datascience-bucket-mm/nasdaq-companylist.csv /tmp/nyse.csv  
  
# Print the result to make sure the transfer worked.
#!cat /tmp/nyse.csv

Copying gs://datascience-bucket-mm/nyse-companylist.csv...
/ [1 files][392.3 KiB/392.3 KiB]                                                
Operation completed over 1 objects/392.3 KiB.                                    
Copying gs://datascience-bucket-mm/nasdaq-companylist.csv...
/ [1 files][462.5 KiB/462.5 KiB]                                                
Operation completed over 1 objects/462.5 KiB.                                    


<h3>  Testing Historical API </h3>

In [0]:

def get_statistics(ticker):
  base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=companyName,symbol,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(symbol=ticker)
  df = pd.read_csv(base_url)
  df['Symbol'] = ticker
  return df

def get_historical_data(ticker, start, end):
  df = get_statistics(ticker)
  return df
  
  return dr.DataReader(ticker, "av-monthly", 
                       datetime.fromordinal(start.toordinal()),  # need to move to av monthly
                       datetime.fromordinal(end.toordinal()),
                       access_key='3K8QCCZYO839KMOU')

def get_date_ranges():
  end_date = date.today()
  start_date = end_date - BDay(60)
  return start_date, end_date

def test():
  start,end = get_date_ranges()
  print('start:{}, end:{}'.format(start_date, end_date))
  print(get_historical_data('AMZN', start_date, end_date))
  
def get_latest_price(symbol):
  base_url = "https://cloud.iexapis.com/stable/stock/{ticker}/quote?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=symbol,close".format(ticker=symbol)
  import requests
  return pd.read_csv(base_url)

def get_latest_price_yahoo(symbol, as_of_date):
  return dr.get_data_yahoo(symbol, as_of_date, as_of_date)[['Close']]
  
    
  

<h3> Reading source data and computing performance </h3>

In [0]:
def get_nyse_df():
  return pd.read_csv('/tmp/nyse.csv', header=0)[['Symbol', 'Name', 'Sector', 'industry']]


In [0]:
def compute_performance(start_dt, end_dt, ticker):
  try:
    import time
    historical_df =  get_historical_data(ticker, start_dt, end_dt)
    print('Historical ok:{}'.format(historical_df.columns))
    
    latest_df = get_latest_price(ticker)
    print ('latest ok:{}'.format(latest_df.columns))
    merged = pd.merge(historical_df, latest_df, how='inner' , left_on="Symbol", right_on="symbol").drop('symbol', axis=1)
    return merged
  except Exception as e:
    print('Exception:{}'.format(str(e)))
    print('Unable to find data for {}:{}'.format(ticker,str(e)))

In [35]:
def find_best_performing(start_dt, end_dt):
  print('Finding Best Performing Stocks between:{}-{}'.format(start_dt, end_dt))
  nyse_df = get_nyse_df()
  symbols = nyse_df['Symbol'].values.tolist()
  print('Now we have to source data for:{}'.format(len(symbols)))
  dfs = (compute_performance(start_dt, end_dt, symbol) for symbol in symbols)
  filtered = (df for df in dfs if df is not None)
  all_data = pd.concat(filtered)
  return pd.merge(nyse_df, all_data, how='inner', on='Symbol' )[['Symbol', 'Name', 'Sector', 'industry', 'companyName','close', 
       'month3ChangePercent', 'month6ChangePercent', 'month1ChangePercent', 'day200MovingAvg', 'day50MovingAvg']]
  

start_dt, end_dt = get_date_ranges()
perf_df = find_best_performing(start_dt, end_dt)#compute_performance(start_dt, end_dt)
print(perf_df.columns)
print(perf_df.shape)

Finding Best Performing Stocks between:2019-04-24 00:00:00-2019-07-17
Now we have to source data for:3524
Exception:'Symbol'
Unable to find data for YI:'Symbol'
Exception:'Symbol'
Unable to find data for PIH:'Symbol'
Exception:'Symbol'
Unable to find data for PIHPP:'Symbol'
Exception:'Symbol'
Unable to find data for TURN:'Symbol'
Exception:'Symbol'
Unable to find data for FLWS:'Symbol'
Exception:'Symbol'
Unable to find data for BCOW:'Symbol'
Exception:'Symbol'
Unable to find data for FCCY:'Symbol'
Exception:'Symbol'
Unable to find data for SRCE:'Symbol'
Exception:'Symbol'
Unable to find data for VNET:'Symbol'
Exception:'Symbol'
Unable to find data for TWOU:'Symbol'
Exception:'Symbol'
Unable to find data for QFIN:'Symbol'
Exception:'Symbol'
Unable to find data for JOBS:'Symbol'
Exception:'Symbol'
Unable to find data for JFK:'Symbol'
Exception:'Symbol'
Unable to find data for JFKKR:'Symbol'
Exception:'Symbol'
Unable to find data for JFKKU:'Symbol'
Exception:'Symbol'
Unable to find data f

ValueError: ignored

In [37]:
# Sorting 
perf_df.sort_values(by=['month3ChangePercent'], inplace=True, ascending=False)
#perf_df.head(20)


NameError: ignored

<h3> Group by sector, to find best performers </h3>

In [0]:
perf_df[['sector', 'month3ChangePercent']].groupby(['sector']).mean().sort_values(by=['month3ChangePercent'], inplace=True, ascending=False)

In [0]:
# Panda test
base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=companyName,symbol,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(symbol='AMZN')
df = pd.read_csv(base_url)
df['symbol'] = 'AMZN'
df.columns
  

Index(['companyName', 'day50MovingAvg', 'day200MovingAvg',
       'month6ChangePercent', 'month3ChangePercent', 'month1ChangePercent',
       'symbol'],
      dtype='object')

In [0]:
get_statistics('AMZN')

,companyName,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent
0,"Amazon.com, Inc.",1.843297,1871.88,1744.88,0.230806,0.066052,0.121094


In [0]:
get_latest_price('AMZN')

,symbol,close
0,AMZN,2009.9


In [0]:
dr.get_data_yahoo('AMZN', date(2019,7,16), date(2019,7,16))


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-07-16,2026.319946,2001.219971,2010.579956,2009.900024,2553900,2009.900024


In [0]:
def get_historical_price(symbol, as_of_date):
  # Leveraging Historical
  import requests
  historical_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/chart/date/{cob}?token=sk_98e397d4bee940488e1f48e9b419508f&chartByDay=true'.format(
                        symbol=symbol, cob=as_of_date.strftime('%Y%m%d'))
  print('Fetching data for {} from:{}'.format(symbol, historical_url))
  return requests.get(historical_url).json()

get_historical_price('AMZN', date(2019,7,16))

Fetching data for AMZN from:https://cloud.iexapis.com/stable/stock/AMZN/chart/date/20190716?token=sk_98e397d4bee940488e1f48e9b419508f&chartByDay=true


[{'change': 0,
  'changeOverTime': 0,
  'changePercent': 0,
  'close': 2009.9,
  'date': '2019-07-16',
  'high': 2026.32,
  'label': 'Jul 16',
  'low': 2001.22,
  'open': 2010.58,
  'uClose': 2009.9,
  'uHigh': 2026.32,
  'uLow': 2001.22,
  'uOpen': 2010.58,
  'uVolume': 2618198,
  'volume': 2618198}]

In [0]:
def get_batch_request():
  import requests
  iex_url = 'https://cloud.iexapis.com/stable/stock/aapl/batch?token=sk_98e397d4bee940488e1f48e9b419508f&types=quote,stats&range=1d&last=1'
  return requests.get(iex_url).json()
get_batch_request()

{'quote': {'avgTotalVolume': 22770576,
  'calculationPrice': 'tops',
  'change': -1.15,
  'changePercent': -0.00562,
  'close': 204.5,
  'closeTime': 1563307200182,
  'companyName': 'Apple, Inc.',
  'delayedPrice': 203.99,
  'delayedPriceTime': 1563392746921,
  'extendedChange': 0.93,
  'extendedChangePercent': 0.00457,
  'extendedPrice': 204.28,
  'extendedPriceTime': 1563370200392,
  'high': 205.091,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexLastUpdated': 1563393597008,
  'iexMarketPercent': 0.033215496439546736,
  'iexRealtimePrice': 203.35,
  'iexRealtimeSize': 20,
  'iexVolume': 436383,
  'lastTradeTime': 1563393597008,
  'latestPrice': 203.35,
  'latestSource': 'IEX real time price',
  'latestTime': '3:59:57 PM',
  'latestUpdate': 1563393597008,
  'latestVolume': 13137934,
  'low': 203.66,
  'marketCap': 935629618000,
  'open': 204.05,
  'openTime': 1563370200444,
  'peRatio': 16.96,
  'previousClose': 204.5,
  'previousVolume': None,
  '

In [49]:
start, end = get_date_ranges()
compute_performance(start, end, 'AMZN')
#get_latest_price('AMZN')

Historical ok:Index(['companyName', 'beta', 'day50MovingAvg', 'day200MovingAvg',
       'month6ChangePercent', 'month3ChangePercent', 'month1ChangePercent',
       'Symbol'],
      dtype='object')
latest ok:Index(['symbol', 'close'], dtype='object')


,companyName,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent,Symbol,close
0,"Amazon.com, Inc.",1.846598,1882.87,1747.01,0.187028,0.077798,0.065678,AMZN,1992.03
